In [311]:
using SymPy
using LinearAlgebra
using Test

In [312]:
import PyCall
PyCall.pyimport_conda("sympy.physics.quantum.spin", "sympy")
import_from(sympy.physics.quantum.spin, (:WignerD, :wignerd), typ=:Any)
PyCall.pyimport_conda("sympy.physics.wigner",       "sympy")
import_from(sympy.physics.wigner)

## Dirac matrives

In [117]:
σx = [Sym(0) 1; 1 0]
σy = [Sym(0) -1im; 1im 0]
σz = [1 Sym(0); 0 -1]
id  = Matrix(I,2,2)
id4  = Matrix(I,4,4)
z = zeros(Int, 2,2);

In [313]:
γ0 = [id z; z -id]
γ1 = [z σx; -σx z]
γ2 = [z σy; -σy z]
γ3 = [z σz; -σz z];
γ = (γ1,γ2,γ3,γ0);

In [314]:
sp(a,b) = a[4]*b[4]-sum(a[1:3].*b[1:3])

sp (generic function with 1 method)

In [315]:
@test sp(γ,γ) == 4*Matrix(I,4,4)

Test Passed

In [316]:
m0,m1,m2,m3 = @vars m0 m1 m2 m3 positive=true
σ1,σ2,σ3 = @vars σ_1 σ_2 σ_3
θhat12, θhat23 = @vars θhat_12 θhat_23
cθhat12, sθhat12 = cos(θhat12), sin(θhat12)
cθhat23, sθhat23 = cos(θhat23), sin(θhat23)
summsq = sum([m0,m1,m2,m3].^2)

  2     2     2     2
m0  + m1  + m2  + m3 

In [227]:
E2, = @vars E2 positive=true
Ep, Em = sqrt(E2+m2), sqrt(E2-m2)

(sqrt(E2 + m2), sqrt(E2 - m2))

## Spinors
 * index `b` for $\Xi_b$
 * index `p` for p

In [228]:
ub⁺ = [1,0,0,0] # dropped sqrt(2m2)
ub⁻ = [0,1,0,0] 
up⁺ = [0,Ep,0,Em]
up⁻ = [-Ep,0,Em,0] 

4-element Array{Sym,1}:
 -sqrt(E2 + m2)
              0
  sqrt(E2 - m2)
              0

## Four vectors in components
calculated in CM frame in KK symmetric configuration

In [230]:
λ1,λ2,λ3 = @vars λ_1 λ_2 λ_3
# 
pb = [0,0,0,2m0^2] # droped /2m0
pp = [0,0,-λ2,(m0^2+m2^2-σ2)] # droped /2m0
pK1 = [λ1*sθhat12,0,λ1*cθhat12, (m0^2+m1^2-σ1)] # droped /2m0
pK3 = [-λ3*sθhat23,0,λ3*cθhat23, (m0^2+m3^2-σ3)] # droped /2m0

4-element Array{Sym,1}:
 -λ_3*sin(θhat_23)
                 0
  λ_3*cos(θhat_23)
 m0^2 + m3^2 - σ_3

In [231]:
term1pp = transpose(up⁺)*γ0*(sp(pK1+pp,γ)+id4*2m0*sqrt(σ3))*ub⁺
term1mm = transpose(up⁻)*γ0*(sp(pK1+pp,γ)+id4*2m0*sqrt(σ3))*ub⁻
term1pp - term1mm == 0

true

In [253]:
term1pp

       _________             
-λ_1*\/ E2 - m2 *sin(θhat_12)

In [254]:
subs(term1pp,
    Em=>λ2^2)

        2             
-λ_1*λ_2 *sin(θhat_12)

In [232]:
term1pm = transpose(up⁺)*γ0*(sp(pK1+pp,γ)+id4*2m0*sqrt(σ3))*ub⁻
term1mp = transpose(up⁻)*γ0*(sp(pK1+pp,γ)+id4*2m0*sqrt(σ3))*ub⁺
simplify(term1mp + term1pm) == 0

true

In [233]:
term1pm

    _________                               _________ /    2          _____   
- \/ E2 - m2 *(-λ_1*cos(θhat_12) + λ_2) + \/ E2 + m2 *\2*m0  + 2*m0*\/ σ_3  + 

  2     2            \
m1  + m2  - σ_1 - σ_2/

In [234]:
subs(term1pm, σ1=>sum([m0,m1,m2,m3].^2)-σ2-σ3)

    _________                               _________ /  2          _____     
- \/ E2 - m2 *(-λ_1*cos(θhat_12) + λ_2) + \/ E2 + m2 *\m0  + 2*m0*\/ σ_3  - m3

2      \
  + σ_3/

In [272]:
subs(subs(subs(term1pm,
    σ1=>summsq-σ2-σ3),
    Em=>λ2^2),
    Ep=>2m0*(pp[4]+2m0*m2))

     /  2               2      \ /  2          _____     2      \      2      
2*m0*\m0  + 2*m0*m2 + m2  - σ_2/*\m0  + 2*m0*\/ σ_3  - m3  + σ_3/ - λ_2 *(-λ_1

                    
*cos(θhat_12) + λ_2)

## Angular functions

In [ ]:
λ(x,y,z) = x^2+y^2+z^2-2x*y-2y*z-2z*x

In [301]:
cosζ221n = 2m2^2*(σ3-m0^2-m3^2)+(m0^2+m2^2-σ2)*(σ1-m2^2-m3^2)
cosζ221d_sq = λ(m0^2,m2^2,σ2)*λ(m3^2,m2^2,σ1)
factor(subs(factor(cosζ221d_sq-cosζ221n^2),σ3=>summsq-σ1-σ2)) # numerator of sin

     2 /  4   2     2   2   2     2   2   2     2   2         2   2   2     2 
-4*m2 *\m0 *m3  - m0 *m1 *m2  - m0 *m1 *m3  + m0 *m1 *σ_1 - m0 *m2 *m3  + m0 *

  2         2   4     2   2         2   2         2             4   2     2   
m2 *σ_2 + m0 *m3  - m0 *m3 *σ_1 - m0 *m3 *σ_2 - m0 *σ_1*σ_2 + m1 *m2  + m1 *m2

4     2   2   2     2   2         2   2         2   2         2             2 
  - m1 *m2 *m3  - m1 *m2 *σ_1 - m1 *m2 *σ_2 + m1 *m3 *σ_2 - m1 *σ_1*σ_2 + m2 *

  2         2             2              2              2\
m3 *σ_1 - m2 *σ_1*σ_2 - m3 *σ_1*σ_2 + σ_1 *σ_2 + σ_1*σ_2 /

In [284]:
ϕ = -factor(λ(λ(σ1,m1^2,m0^2),λ(σ2,m2^2,m0^2),λ(summsq-σ1-σ2,m3^2,m0^2)))/(16m0^2)

    4   2     2   2   2     2   2   2     2   2         2   2   2     2   2   
- m0 *m3  + m0 *m1 *m2  + m0 *m1 *m3  - m0 *m1 *σ_1 + m0 *m2 *m3  - m0 *m2 *σ_

      2   4     2   2         2   2         2             4   2     2   4     
2 - m0 *m3  + m0 *m3 *σ_1 + m0 *m3 *σ_2 + m0 *σ_1*σ_2 - m1 *m2  - m1 *m2  + m1

2   2   2     2   2         2   2         2   2         2             2   2   
 *m2 *m3  + m1 *m2 *σ_1 + m1 *m2 *σ_2 - m1 *m3 *σ_2 + m1 *σ_1*σ_2 - m2 *m3 *σ_

      2             2              2              2
1 + m2 *σ_1*σ_2 + m3 *σ_1*σ_2 - σ_1 *σ_2 - σ_1*σ_2 

In [279]:
cosθ23n = 2σ1*(σ3-m1^2-m2^2)-(σ1+m2^2-m3^2)*(m0^2-σ1-m1^2)
cosθ23d_sq = λ(m0^2,m1^2,σ1)*λ(σ1,m2^2,m3^2)
factor(subs(factor(cosθ23d_sq-cosθ23n^2),σ3=>summsq-σ1-σ2)) # numerator of sin

       /  4   2     2   2   2     2   2   2     2   2         2   2   2     2 
-4*σ_1*\m0 *m3  - m0 *m1 *m2  - m0 *m1 *m3  + m0 *m1 *σ_1 - m0 *m2 *m3  + m0 *

  2         2   4     2   2         2   2         2             4   2     2   
m2 *σ_2 + m0 *m3  - m0 *m3 *σ_1 - m0 *m3 *σ_2 - m0 *σ_1*σ_2 + m1 *m2  + m1 *m2

4     2   2   2     2   2         2   2         2   2         2             2 
  - m1 *m2 *m3  - m1 *m2 *σ_1 - m1 *m2 *σ_2 + m1 *m3 *σ_2 - m1 *σ_1*σ_2 + m2 *

  2         2             2              2              2\
m3 *σ_1 - m2 *σ_1*σ_2 - m3 *σ_1*σ_2 + σ_1 *σ_2 + σ_1*σ_2 /

In [302]:
cosζ231n = 2m2^2*(m3^2+m1^2-σ2)+(σ3-m2^2-m1^2)*(σ1-m2^2-m3^2)
cosζ231d_sq = λ(σ3,m1^2,m2^2)*λ(σ1,m2^2,m3^2)
factor(subs(factor(cosζ231d_sq-cosζ231n^2),σ3=>summsq-σ1-σ2))

     2 /  4   2     2   2   2     2   2   2     2   2         2   2   2     2 
-4*m2 *\m0 *m3  - m0 *m1 *m2  - m0 *m1 *m3  + m0 *m1 *σ_1 - m0 *m2 *m3  + m0 *

  2         2   4     2   2         2   2         2             4   2     2   
m2 *σ_2 + m0 *m3  - m0 *m3 *σ_1 - m0 *m3 *σ_2 - m0 *σ_1*σ_2 + m1 *m2  + m1 *m2

4     2   2   2     2   2         2   2         2   2         2             2 
  - m1 *m2 *m3  - m1 *m2 *σ_1 - m1 *m2 *σ_2 + m1 *m3 *σ_2 - m1 *σ_1*σ_2 + m2 *

  2         2             2              2              2\
m3 *σ_1 - m2 *σ_1*σ_2 - m3 *σ_1*σ_2 + σ_1 *σ_2 + σ_1*σ_2 /

In [303]:
cosθhat12n = (m0^2+m1^2-σ1)*(m0^2+m2^2-σ2) - 2m0^2*(σ3-m1^2-m2^2)
cosθhat12d_sq = λ(m0^2,m2^2,σ2)*λ(m0^2,σ1,m1^2)
factor(subs(factor(cosθhat12d_sq-cosθhat12n^2),σ3=>summsq-σ1-σ2)) # numerator of sin

     2 /  4   2     2   2   2     2   2   2     2   2         2   2   2     2 
-4*m0 *\m0 *m3  - m0 *m1 *m2  - m0 *m1 *m3  + m0 *m1 *σ_1 - m0 *m2 *m3  + m0 *

  2         2   4     2   2         2   2         2             4   2     2   
m2 *σ_2 + m0 *m3  - m0 *m3 *σ_1 - m0 *m3 *σ_2 - m0 *σ_1*σ_2 + m1 *m2  + m1 *m2

4     2   2   2     2   2         2   2         2   2         2             2 
  - m1 *m2 *m3  - m1 *m2 *σ_1 - m1 *m2 *σ_2 + m1 *m3 *σ_2 - m1 *σ_1*σ_2 + m2 *

  2         2             2              2              2\
m3 *σ_1 - m2 *σ_1*σ_2 - m3 *σ_1*σ_2 + σ_1 *σ_2 + σ_1*σ_2 /

### Substiture expression for the amplitude
$$
A_{\nu,\lambda} = d_{\nu,\lambda}^{1/2}(\hat\theta_{1(2)}+\theta_{23}-\zeta^2_{2(1)})
$$

In [248]:
α,β,δ = @vars α β δ

(α, β, δ)

In [304]:
expand(sympy.expand_trig(cos(α+β-δ)))

-sin(α)*sin(β)*cos(δ) + sin(α)*sin(δ)*cos(β) + sin(β)*sin(δ)*cos(α) + cos(α)*c
os(β)*cos(δ)

In [299]:
# expression for the half angle sum (α+β-δ)/2
v = simplify(subs(subs(subs(subs(subs(subs(expand(sympy.expand_trig(cos(α+β-δ))),
    sin(α)=>2m0*ϕ),
    cos(α)=>cosθhat12n),
    sin(β)=>2sqrt(σ1)*ϕ),
    cos(β)=>cosθ23n),
    sin(δ)=>2m2*ϕ),
    cos(δ)=>cosζ221n))

                                                                              
          /      /  2     2      \   /    2     2      \ /  2     2      \\ / 
- 4*m0*m2*\2*σ_1*\m1  + m2  - σ_3/ - \- m0  + m1  + σ_1/*\m2  - m3  + σ_1//*\-

                                                                              
   4   2     2   2   2     2   2   2     2   2         2   2   2     2   2    
 m0 *m3  + m0 *m1 *m2  + m0 *m1 *m3  - m0 *m1 *σ_1 + m0 *m2 *m3  - m0 *m2 *σ_2

                                                                              
     2   4     2   2         2   2         2             4   2     2   4     2
 - m0 *m3  + m0 *m3 *σ_1 + m0 *m3 *σ_2 + m0 *σ_1*σ_2 - m1 *m2  - m1 *m2  + m1 

                                                                              
   2   2     2   2         2   2         2   2         2             2   2    
*m2 *m3  + m1 *m2 *σ_1 + m1 *m2 *σ_2 - m1 *m3 *σ_2 + m1 *σ_1*σ_2 - m2 *m3 *σ_1

                                                

half angle (α+β-δ)
$$
\cos\frac{\theta}{2} = \sqrt{\frac{1+\cos\theta}{2}}
$$
should factorize

In [300]:
# factor(simplify(v + cosθhat12d_sq*cosθ23n_sq*cosζ221n_sq))

  16   8       16   6   2       16   6           16   4   4       16   4   2  
m0  *m2  - 4*m0  *m2 *m3  - 4*m0  *m2 *σ_1 + 6*m0  *m2 *m3  + 4*m0  *m2 *m3 *σ

         16   4    2       16   2   6       16   2   4           16   2   2   
_1 + 6*m0  *m2 *σ_1  - 4*m0  *m2 *m3  + 4*m0  *m2 *m3 *σ_1 + 4*m0  *m2 *m3 *σ_

 2       16   2    3     16   8       16   6           16   4    2       16   
1  - 4*m0  *m2 *σ_1  + m0  *m3  - 4*m0  *m3 *σ_1 + 6*m0  *m3 *σ_1  - 4*m0  *m3

2    3     16    4       14   2   8        14   2   6   2        14   2   6   
 *σ_1  + m0  *σ_1  - 4*m0  *m1 *m2  + 16*m0  *m1 *m2 *m3  + 16*m0  *m1 *m2 *σ_

         14   2   4   4        14   2   4   2            14   2   4    2      
1 - 24*m0  *m1 *m2 *m3  - 16*m0  *m1 *m2 *m3 *σ_1 - 24*m0  *m1 *m2 *σ_1  + 16*

  14   2   2   6        14   2   2   4            14   2   2   2    2        1
m0  *m1 *m2 *m3  - 16*m0  *m1 *m2 *m3 *σ_1 - 16*m0  *m1 *m2 *m3 *σ_1  + 16*m0 

4   2   2    3       14   2   8        14   2 